***Note:** The code might be interrupted due to a large number of requests to the web. If an error occurs, please try running it again.*

In [ ]:
# Import necessary libraries
import requests
import time
import random
import pandas as pd
from tqdm import tqdm

In [2]:
# Define the headers for the request
headers1 = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
'Accept':'application/json, text/plain, */*',
'Accept-Language':'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7',
'Referer':'https://tiki.vn/nha-sach-tiki/c8322',
'x-guest-token':'fTjEIaxPWsDzNgK7JoA2Ul3mdptFc1yR',
'Connection':'keep-alive',
'TE': 'Trailers'
}

In [3]:
# Define the parameters for the request
params1 = {
    'limit': '55',
    'include':'advertisement',
    'aggregations':'2',
    'verson':'home-persionalized',
    'trackity_id':'85b9437a-0f42-4a39-bca8-7ea0c3f80ba3',
    'category':'8322',
    'page':'1',
    'urlKey':'nha-sach-tiki'
}

In [4]:
# Function to extract the necessary items from the JSON response
def scrap_id(record):
    d = dict()
    d['product_id'] = record.get('id')
    d['seller_id'] = record.get('seller_id')
    d['seller_product_id'] = record.get('seller_product_id')
    return d
    

In [5]:
# Create an empty list to store the items
product_id = []
# Loop through multiple pages to collect data
for i in range(1,100):
    params1['page'] = i
    response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers1, params=params1)
    if response.status_code == 200:
        print('Collected sucessfuly page:', i)
        for record in response.json().get('data'):
            product_id.append(scrap_id(record))
    time.sleep(random.randrange(0,2))

Collected sucessfuly page: 1
Collected sucessfuly page: 2
Collected sucessfuly page: 3
Collected sucessfuly page: 4
Collected sucessfuly page: 5
Collected sucessfuly page: 6
Collected sucessfuly page: 7
Collected sucessfuly page: 8
Collected sucessfuly page: 9
Collected sucessfuly page: 10
Collected sucessfuly page: 11
Collected sucessfuly page: 12
Collected sucessfuly page: 13
Collected sucessfuly page: 14
Collected sucessfuly page: 15
Collected sucessfuly page: 16
Collected sucessfuly page: 17
Collected sucessfuly page: 18
Collected sucessfuly page: 19
Collected sucessfuly page: 20
Collected sucessfuly page: 21
Collected sucessfuly page: 22
Collected sucessfuly page: 23
Collected sucessfuly page: 24
Collected sucessfuly page: 25
Collected sucessfuly page: 26
Collected sucessfuly page: 27
Collected sucessfuly page: 28
Collected sucessfuly page: 29
Collected sucessfuly page: 30
Collected sucessfuly page: 31
Collected sucessfuly page: 32
Collected sucessfuly page: 33
Collected sucessful

In [6]:
# Convert the list into a DataFrame
productkey = pd.DataFrame(product_id)
productkey

,product_id,seller_id,seller_product_id
0,275025208,286446,275025209
1,190843478,10684,190843479
2,25728855,9988,175540726
3,52789367,1,52789368
4,23416722,1,23416723
...,...,...,...
2025,274948798,1,274948801
2026,24490036,1,24490037
2027,273314953,1,273314954
2028,275137720,53660,275137721


In [7]:
productkey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2030 entries, 0 to 2029
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   product_id         2030 non-null   int64
 1   seller_id          2030 non-null   int64
 2   seller_product_id  2030 non-null   int64
dtypes: int64(3)
memory usage: 47.7 KB


In [8]:
# Drop duplicate entries based on 'product_id'
productkey_clean = productkey.drop_duplicates('product_id')
productkey_clean

,product_id,seller_id,seller_product_id
0,275025208,286446,275025209
1,190843478,10684,190843479
2,25728855,9988,175540726
3,52789367,1,52789368
4,23416722,1,23416723
...,...,...,...
2025,274948798,1,274948801
2026,24490036,1,24490037
2027,273314953,1,273314954
2028,275137720,53660,275137721


In [9]:
# Define the headers for the detailed product information request
headers2 = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
'Accept':'application/json, text/plain, */*',
'Accept-Language':'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7',
'Referer':'https://tiki.vn/ly-thuyet-tro-choi-p271379451.html?spid=271379452',
'x-guest-token':'fTjEIaxPWsDzNgK7JoA2Ul3mdptFc1yR',
'Connection':'keep-alive',
'TE': 'Trailers'
}

In [10]:
# Define the parameters for the detailed product information request
params2 = {
    'platform':'web',
    'spid':'271379452',
    'verson':'3'
}

In [11]:
# Extract the necessary items from the DataFrame
seller_product_id = productkey_clean.seller_product_id.to_list()
product_id = productkey_clean.product_id.to_list()
seller_id = productkey_clean.seller_id.to_list()

In [12]:
# Function to extract detailed product information
def scrap_productinfo(json):
    d = dict()
    d['product_id'] = json.get('id')
    d['name'] = json.get('name')
    authors = json.get('authors')
    d['author'] = authors[0].get('name') if authors else None
    d['quantity_sold'] = json.get('all_time_quantity_sold')
    categories = json.get('breadcrumbs')
    d['categories'] = categories[2].get('name') if categories else None
    subcategories = json.get('categories')
    if subcategories is not None:    
        d['subcategories'] = subcategories.get('name') 
    else:
        d['subcategories'] = None
    current_seller  = json.get('current_seller')
    if current_seller is not None:    
        d['seller_id'] = current_seller.get('id')
        d['seller_name'] = current_seller.get('name')
        d['seller_link'] = current_seller.get('link')
        d['seller_product_id'] = current_seller.get('product_id')
    else:
        d['seller_id'] = None
        d['seller_name'] = None
        d['seller_link'] = None
        d['seller_product_id'] = None
    d['inventory_status'] = json.get('inventory_status')
    d['list_price'] = json.get('list_price')
    d['current_price'] = json.get('price')
    d['discount'] = json.get('discount')
    d['discount_rate'] = json.get('discount_rate')
    d['short_description'] = json.get('short_description')
    d['short_url'] = json.get('short_url')
    d['rating_average'] = json.get('rating_average')
    return d

In [13]:
# Loop through each product to collect detailed information
product_data=[]
for pid,spid in tqdm(zip(product_id,seller_product_id), total = len(product_id)):
    params2['spid'] = spid
    response = requests.get('https://tiki.vn/api/v2/products/{}'.format(pid), headers=headers2, params=params2)
    if response.status_code == 200:
        product_data.append(scrap_productinfo(response.json()))
    time.sleep(random.randrange(0,2))

product_info = pd.DataFrame(product_data)
product_info

  0%|          | 0/2008 [00:00<?, ?it/s]

100%|██████████| 2008/2008 [26:36<00:00,  1.26it/s]


,product_id,name,author,quantity_sold,categories,subcategories,seller_id,seller_name,seller_link,seller_product_id,inventory_status,list_price,current_price,discount,discount_rate,short_description,short_url,rating_average
0,275025208,Sách Combo 2 Cuốn : Tư Duy Ngược + Tư Duy Mở (...,Nguyễn Anh Dũng,2129.0,Sách kỹ năng sống,Sách tư duy - Kỹ năng sống,286446,TIỆM SÁCH KHAI TÂM,https://tiki.vn/cua-hang/tiem-sach-khai-tam,275025209,available,276000,138000,138000,50,THÔNG TIN TÁC PHẨMTên tác phẩm: Combo 2 Cuốn T...,https://tiki.vn/product-p275025208.html?spid=2...,4.8
1,190843478,COMBO 4 - (TRÍ TUỆ CỦA NGƯỜI XƯA - ĐẠO LÝ NGƯỜ...,None,1954.0,Sách Kiến Thức Tổng Hợp,Triết Học phương Đông,10684,SBOOKS OFFICIAL,https://tiki.vn/cua-hang/sbooks,190843479,available,582000,321000,261000,45,TRÍ TUỆ của NGƯỜI XƯATrí tuệ và mưu kế của ngư...,https://tiki.vn/product-p190843478.html?spid=1...,4.8
2,25728855,Kinh Dịch (Ngô Tất Tố),Ngô Tất Tố,66.0,Sách Kiến Thức Tổng Hợp,Kinh Dịch Học,9988,AHABOOKS,https://tiki.vn/cua-hang/ahabooks,175540726,available,193250,193250,0,0,"Trong lĩnh vực khảo cứu, dịch thuật, Ngô Tất T...",https://tiki.vn/product-p25728855.html?spid=17...,5.0
3,52789367,Nhà Giả Kim (Tái Bản 2020),Paulo Coelho,35050.0,Sách văn học,Tác Phẩm Kinh Điển Nước Ngoài,1,Tiki Trading,https://tiki.vn/cua-hang/tiki-trading,52789368,available,79000,63200,15800,20,Sơ lược về tác phẩmTất cả những trải nghiệm tr...,https://tiki.vn/product-p52789367.html?spid=52...,5.0
4,23416722,How The Body Works - Hiểu Hết Về Cơ Thể,Phạm Hằng Nguyên,6386.0,Sách Y Học,Nhà Sách Tiki,1,Tiki Trading,https://tiki.vn/cua-hang/tiki-trading,23416723,available,300000,240000,60000,20,Truyền thông ngập tràn những khám phá và những...,https://tiki.vn/product-p23416722.html?spid=23...,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003,274948798,Nhóc Maruko Tập 8 [Tặng Kèm Set Postcard Polar...,None,98.0,"Truyện Tranh, Manga, Comic","Truyện Tranh, Manga, Comic",1,Tiki Trading,https://tiki.vn/cua-hang/tiki-trading,274948801,available,40000,38500,1500,4,"Tác giả Sakura chia sẻ: “Dạo này, Nagasawa bắt...",https://tiki.vn/product-p274948798.html?spid=2...,5.0
2004,24490036,Tại Sao Phương Tây Vượt Trội,Ian Morris,3910.0,Sách Kiến Thức Tổng Hợp,Kiến Thức Bách Khoa,1,Tiki Trading,https://tiki.vn/cua-hang/tiki-trading,24490037,available,468000,291400,176600,38,Tại Sao Phương Tây Vượt TrộiBằng nghiên cứu sâ...,https://tiki.vn/product-p24490036.html?spid=24...,4.7
2005,273314953,Cất Cho Tôi Những Ngày Xanh Nắng Hạ,Minh,157.0,Sách văn học,Truyện ngắn - Tản văn - Tạp Văn Việt Nam,1,Tiki Trading,https://tiki.vn/cua-hang/tiki-trading,273314954,available,109000,67600,41400,38,Tuổi trẻ không chỉ được lấp đầy bằng niềm vui ...,https://tiki.vn/product-p273314953.html?spid=2...,4.7
2006,275137720,Chuyên Đề Học Tập Lịch Sử 10 (Chân Trời Sáng T...,nhieu tac gia,27.0,Sách Giáo Khoa - Giáo Trình,Root,53660,Nhà sách Fahasa,https://tiki.vn/cua-hang/nha-sach-fahasa,275137721,available,9000,9000,0,0,Chuyên Đề Học Tập Lịch Sử 10 (Chân Trời Sá...,https://tiki.vn/product-p275137720.html?spid=2...,5.0


In [14]:
product_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008 entries, 0 to 2007
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_id         2008 non-null   int64  
 1   name               2008 non-null   object 
 2   author             1584 non-null   object 
 3   quantity_sold      1998 non-null   float64
 4   categories         2008 non-null   object 
 5   subcategories      2008 non-null   object 
 6   seller_id          2008 non-null   int64  
 7   seller_name        2008 non-null   object 
 8   seller_link        2008 non-null   object 
 9   seller_product_id  2008 non-null   object 
 10  inventory_status   2008 non-null   object 
 11  list_price         2008 non-null   int64  
 12  current_price      2008 non-null   int64  
 13  discount           2008 non-null   int64  
 14  discount_rate      2008 non-null   int64  
 15  short_description  2008 non-null   object 
 16  short_url          2008 

In [15]:
product_info.head()

,product_id,name,author,quantity_sold,categories,subcategories,seller_id,seller_name,seller_link,seller_product_id,inventory_status,list_price,current_price,discount,discount_rate,short_description,short_url,rating_average
0,275025208,Sách Combo 2 Cuốn : Tư Duy Ngược + Tư Duy Mở (...,Nguyễn Anh Dũng,2129.0,Sách kỹ năng sống,Sách tư duy - Kỹ năng sống,286446,TIỆM SÁCH KHAI TÂM,https://tiki.vn/cua-hang/tiem-sach-khai-tam,275025209,available,276000,138000,138000,50,THÔNG TIN TÁC PHẨMTên tác phẩm: Combo 2 Cuốn T...,https://tiki.vn/product-p275025208.html?spid=2...,4.8
1,190843478,COMBO 4 - (TRÍ TUỆ CỦA NGƯỜI XƯA - ĐẠO LÝ NGƯỜ...,None,1954.0,Sách Kiến Thức Tổng Hợp,Triết Học phương Đông,10684,SBOOKS OFFICIAL,https://tiki.vn/cua-hang/sbooks,190843479,available,582000,321000,261000,45,TRÍ TUỆ của NGƯỜI XƯATrí tuệ và mưu kế của ngư...,https://tiki.vn/product-p190843478.html?spid=1...,4.8
2,25728855,Kinh Dịch (Ngô Tất Tố),Ngô Tất Tố,66.0,Sách Kiến Thức Tổng Hợp,Kinh Dịch Học,9988,AHABOOKS,https://tiki.vn/cua-hang/ahabooks,175540726,available,193250,193250,0,0,"Trong lĩnh vực khảo cứu, dịch thuật, Ngô Tất T...",https://tiki.vn/product-p25728855.html?spid=17...,5.0
3,52789367,Nhà Giả Kim (Tái Bản 2020),Paulo Coelho,35050.0,Sách văn học,Tác Phẩm Kinh Điển Nước Ngoài,1,Tiki Trading,https://tiki.vn/cua-hang/tiki-trading,52789368,available,79000,63200,15800,20,Sơ lược về tác phẩmTất cả những trải nghiệm tr...,https://tiki.vn/product-p52789367.html?spid=52...,5.0
4,23416722,How The Body Works - Hiểu Hết Về Cơ Thể,Phạm Hằng Nguyên,6386.0,Sách Y Học,Nhà Sách Tiki,1,Tiki Trading,https://tiki.vn/cua-hang/tiki-trading,23416723,available,300000,240000,60000,20,Truyền thông ngập tràn những khám phá và những...,https://tiki.vn/product-p23416722.html?spid=23...,4.8


In [16]:
category_count = product_info.groupby('categories').size().reset_index(name = 'count')
category_count

,categories,count
0,1980Books,1
1,Bookmark,1
2,Business & Economics,2
3,Bút - Viết các loại,7
4,Dictionary,2
5,Dụng Cụ Học Sinh,8
6,Dụng Cụ Văn Phòng,4
7,Education - Teaching,142
8,Fiction - Literature,1
9,Flashcards,2


In [17]:
# Define the parameters for the product review request
params3 = {
    'limit':'5',
    'include':'comments,contribute_info,attribute_vote_summary',
    'sort':'score|desc,id|desc,stars|all',
    'page':'1',
    'spid':'271379452',
    'product_id':'271379451',
    'seller_id':'1'
}

In [18]:
# Function to extract product review information
def product_review(json,pid,sid):
    d = dict()
    d['product_id'] = pid
    d['seller_id'] = sid
    review = json.get('stars')
    if review is not None:
        for i in range(1,6):
            d['{} star'.format(i)] = review['{}'.format(i)].get('count')
    else:
        for i in range(1,6):
            d['{} star'.format(i)] = None
    d['rating_average'] = json.get('rating_average')
    d['reviews_count'] = json.get('reviews_count')
    return d
    

In [19]:
# Loop through each product to collect review data
review = []
for pid,sid,spid in tqdm(zip(product_id,seller_id,seller_product_id), total=len(product_id)):
    params3['product_id'] = pid
    params3['seller_id'] = sid
    params3['spid'] = spid
    response = requests.get('https://tiki.vn/api/v2/reviews',headers=headers2, params=params3)
    if response.status_code == 200:
        json = response.json()
        review.append(product_review(json,pid,sid))
    time.sleep(random.randrange(0,2))

review = pd.DataFrame(review)
review.head()


100%|██████████| 2008/2008 [31:21<00:00,  1.07it/s]


,product_id,seller_id,1 star,2 star,3 star,4 star,5 star,rating_average,reviews_count
0,275025208,286446,2,0,2,11,83,4.8,98
1,190843478,10684,1,1,2,15,107,4.8,126
2,25728855,9988,0,0,0,0,6,5.0,6
3,52789367,1,34,20,62,433,5374,4.9,5923
4,23416722,1,15,13,21,102,873,4.8,1024


In [20]:
# Drop duplicate columns from the review DataFrame
product_review = review.drop(columns=['rating_average'])


In [21]:
# Identify duplicate
product_duplicates = product_info.groupby(['product_id','name','short_url']).size().reset_index(name = 'count')
product_duplicates = product_duplicates[product_duplicates['count']>1]
print(product_duplicates)

Empty DataFrame
Columns: [product_id, name, short_url, count]
Index: []


In [22]:
# Join data from the product and review table
book_data = pd.merge(product_info, product_review, on='product_id', how='inner')

In [23]:
book_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008 entries, 0 to 2007
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_id         2008 non-null   int64  
 1   name               2008 non-null   object 
 2   author             1584 non-null   object 
 3   quantity_sold      1998 non-null   float64
 4   categories         2008 non-null   object 
 5   subcategories      2008 non-null   object 
 6   seller_id_x        2008 non-null   int64  
 7   seller_name        2008 non-null   object 
 8   seller_link        2008 non-null   object 
 9   seller_product_id  2008 non-null   object 
 10  inventory_status   2008 non-null   object 
 11  list_price         2008 non-null   int64  
 12  current_price      2008 non-null   int64  
 13  discount           2008 non-null   int64  
 14  discount_rate      2008 non-null   int64  
 15  short_description  2008 non-null   object 
 16  short_url          2008 

In [24]:
book_data.describe()

,product_id,quantity_sold,seller_id_x,list_price,current_price,discount,discount_rate,rating_average,seller_id_y,1 star,2 star,3 star,4 star,5 star,reviews_count
count,2.008000e+03,1998.000000,2008.000000,2.008000e+03,2.008000e+03,2008.000000,2008.000000,2008.000000,2008.000000,2008.000000,2008.000000,2008.000000,2008.000000,2008.000000,2008.000000
mean,1.794161e+08,2730.301802,36617.183267,1.656771e+05,1.272645e+05,38412.595618,20.573705,4.347211,36617.183267,2.267928,1.474602,3.923805,23.125498,224.731574,255.523406
std,9.800917e+07,20903.942247,75379.406929,1.531180e+05,1.198141e+05,48947.117405,14.140111,1.531874,75379.406929,6.501077,4.712814,12.103633,65.683598,624.712217,708.677442
min,3.326260e+05,1.000000,1.000000,2.000000e+03,2.000000e+03,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.528983e+07,56.000000,1.000000,8.500000e+04,6.497500e+04,4575.000000,7.000000,4.800000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000
50%,1.997954e+08,349.000000,1.000000,1.310000e+05,9.864000e+04,29725.000000,23.000000,5.000000,1.000000,0.000000,0.000000,0.000000,2.000000,22.000000,24.000000
75%,2.744039e+08,1869.750000,11216.000000,1.990000e+05,1.490000e+05,50820.000000,30.000000,5.000000,11216.000000,2.000000,1.000000,2.000000,15.000000,168.000000,187.250000
max,2.760196e+08,898361.000000,354790.000000,1.928000e+06,1.480000e+06,744000.000000,84.000000,5.000000,354790.000000,78.000000,57.000000,151.000000,924.000000,12221.000000,13342.000000


In [25]:
# Convert data to Excel
book_data.to_excel("Book_data.xlsx", index=False)

In [26]:
book_data.to_excel("Book_data_backup.xlsx", index=False)